# Run a job to get optimisation results

## Configure Watson Machine Learning client

In [1]:
# Config from previous deployment
deployment_uid = '541cc38c-f63c-4319-b45f-afde3ccef771'
space_uid = 'ba8c7337-02aa-4048-96fd-ffdd2ab82e76'

In [2]:
## Create a WML v4 client
from ibm_watson_machine_learning import APIClient

wml_credentials = {
      "apikey": "<api key>",
      "url": "https://us-south.ml.cloud.ibm.com"
}

client = APIClient(wml_credentials)
print(client.version)

1.0.45


In [3]:
# Set client's default space
client.set.default_space(space_uid)

'SUCCESS'

## Prepare input data

In [4]:
import pandas as pd 
  
# initialize list of lists 
input_params = pd.DataFrame([{"axe_strength": 1, "lc_strength": 1, "ma_strength": 1, "build_time_limit_seconds": 2_592_000}])
print(input_params)

   axe_strength  lc_strength  ma_strength  build_time_limit_seconds
0             1            1            1                   2592000


## Create optimisation job

In [5]:
# Prepare inputs and outputs expectations for solver

# Using inline tabular data
# https://dataplatform.cloud.ibm.com/docs/content/DO/WML_Deployment/ModelIODataDefn.html?context=wdp&audience=wdp
solve_payload = {
    client.deployments.DecisionOptimizationMetaNames.INPUT_DATA: [{
      "id":"params.csv", # This is required by my model
      "values" : input_params
    }],
    client.deployments.DecisionOptimizationMetaNames.OUTPUT_DATA: [{
        "id":".*\.csv" # My model is configured to output CSVs for results
    }]
}

job_details = client.deployments.create_job(deployment_uid, solve_payload)
job_uid = client.deployments.get_job_uid(job_details)

print(job_uid)

75ffcc3e-8e50-4773-bed8-a55c46fe2d78


## Wait for job to complete

In [6]:
# Job can take a while to run - a machine with CPLEX needs to start first
from time import sleep

while job_details['entity']['decision_optimization']['status']['state'] not in ['completed', 'failed', 'canceled']:
    print(job_details['entity']['decision_optimization']['status']['state'] + '...')
    sleep(5)
    job_details=client.deployments.get_job_details(job_uid)

print( job_details['entity']['decision_optimization']['status']['state'])

queued...
completed


## Print results

In [7]:
from pprint import pprint

pprint(job_details['entity']['decision_optimization']['output_data'])

[{'fields': ['Name', 'Value'],
  'id': 'kpis.csv',
  'values': [['Total negative attack strength', -682455],
             ['Barracks build time', 1799760],
             ['Hall of Order build time', 133200]]},
 {'fields': ['axe',
             'food',
             'lc',
             'ma',
             'ram',
             'serk',
             'time_in_days',
             'time_in_seconds'],
  'id': 'solution.csv',
  'values': [[4814, 20596, 1664, 1442, 250, 111, 22.39, 1934400],
             [4814, 20596, 1664, 1442, 250, 111, 22.39, 1934400],
             [4985, 20596, 1725, 1491, 250, 1, 21.55, 1861800],
             [4978, 20580, 1724, 1490, 250, 1, 21.53, 1860360],
             [4807, 20580, 1663, 1441, 250, 111, 22.37, 1932960],
             [4807, 20580, 1663, 1441, 250, 111, 22.37, 1932960]]}]


### Full output

In [8]:
job_details

{'entity': {'decision_optimization': {'input_data': [{'fields': ['axe_strength',
      'lc_strength',
      'ma_strength',
      'build_time_limit_seconds'],
     'id': 'params.csv',
     'values': [[1, 1, 1, 2592000]]}],
   'output_data': [{'fields': ['Name', 'Value'],
     'id': 'kpis.csv',
     'values': [['Total negative attack strength', -682455],
      ['Barracks build time', 1799760],
      ['Hall of Order build time', 133200]]},
    {'fields': ['axe',
      'food',
      'lc',
      'ma',
      'ram',
      'serk',
      'time_in_days',
      'time_in_seconds'],
     'id': 'solution.csv',
     'values': [[4814, 20596, 1664, 1442, 250, 111, 22.39, 1934400],
      [4814, 20596, 1664, 1442, 250, 111, 22.39, 1934400],
      [4985, 20596, 1725, 1491, 250, 1, 21.55, 1861800],
      [4978, 20580, 1724, 1490, 250, 1, 21.53, 1860360],
      [4807, 20580, 1663, 1441, 250, 111, 22.37, 1932960],
      [4807, 20580, 1663, 1441, 250, 111, 22.37, 1932960]]}],
   'output_data_references': [],
